In [1]:
import sys
import pandas as pd
import numpy as np

# loading train.csv file, encoding as big 5
data = pd.read_csv('date/train.csv', encoding='big5')
data.head(5)

,???,????,????.1,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,???,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,???,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,???,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,???,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,???,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5


preprocessing

In [2]:
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
# convert data  to 2 dimesion
raw_data = data.to_numpy()
print(raw_data)

[['14' '14' '14' ... '15' '15' '15']
 ['1.8' '1.8' '1.8' ... '1.8' '1.8' '1.8']
 ['0.51' '0.41' '0.39' ... '0.35' '0.36' '0.32']
 ...
 ['36' '55' '72' ... '118' '100' '105']
 ['1.9' '2.4' '1.9' ... '1.5' '2' '2']
 ['0.7' '0.8' '1.8' ... '1.6' '1.8' '2']]


extract features

In [3]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 20*24])
    for day in range(20):
        sample[:, day * 24 : (day+1)*24] = raw_data[18*(20*month+day): 18*(20*month+day+1), :]
    month_data[month] = sample
print(month_data)

x = np.empty([12 * 471, 18 * 9], dtype=float)
y = np.empty([12 * 471, 1], dtype=float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month*471 + day*24 + hour, :] = month_data[month][:, day*24 + hour: day*24 + hour + 9].reshape(1, -1)  # vector
            y[month*471 + day*24 + hour, 0] = month_data[month][9, day*24 + hour + 9] # value
print(x)
print(y)

{0: array([[14.  , 14.  , 14.  , ..., 14.  , 13.  , 13.  ],
       [ 1.8 ,  1.8 ,  1.8 , ...,  1.8 ,  1.8 ,  1.8 ],
       [ 0.51,  0.41,  0.39, ...,  0.34,  0.41,  0.43],
       ...,
       [35.  , 79.  ,  2.4 , ..., 48.  , 63.  , 53.  ],
       [ 1.4 ,  1.8 ,  1.  , ...,  1.1 ,  1.9 ,  1.9 ],
       [ 0.5 ,  0.9 ,  0.6 , ...,  1.2 ,  1.2 ,  1.3 ]]), 1: array([[ 15.  ,  14.  ,  14.  , ...,   8.4 ,   8.  ,   7.6 ],
       [  1.8 ,   1.8 ,   1.7 , ...,   1.7 ,   1.7 ,   1.7 ],
       [  0.27,   0.26,   0.25, ...,   0.36,   0.35,   0.32],
       ...,
       [113.  , 109.  , 104.  , ...,  72.  ,  65.  ,  69.  ],
       [  2.3 ,   2.2 ,   2.6 , ...,   1.9 ,   2.9 ,   1.5 ],
       [  2.5 ,   2.2 ,   2.2 , ...,   0.9 ,   1.6 ,   1.1 ]]), 2: array([[ 18.  ,  18.  ,  18.  , ...,  14.  ,  13.  ,  13.  ],
       [  1.8 ,   1.8 ,   1.8 , ...,   1.8 ,   1.8 ,   1.8 ],
       [  0.39,   0.36,   0.4 , ...,   0.42,   0.47,   0.49],
       ...,
       [103.  , 128.  , 115.  , ...,  60.  ,  94.  ,  53

normalize

In [25]:
mean_x = np.mean(x, axis=0)
std_x = np.std(x, axis=0)
for i in range(len(x)):
    for j in range(len(x[0])):
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
print(x)


[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.38950555 -0.10906991
   0.07797893]]


split training data into "train_set" and "validation_set"


In [27]:
import math

x_train_set = x[: math.floor(len(x)*0.8), :]
y_train_set = y[: math.floor(len(y)*0.8), :]
x_validation_set = x[math.floor(len(x)*0.8):, :]
y_validation_set = y[math.floor(len(x)*0.8):, :]


train model

In [29]:
dim = 18 * 9 + 1
w = np.zeros([dim, 1])
x = np.concatenate((np.ones([12*471, 1]), x), axis=1).astype(float)
learning_rate = 100
iter_time = 1000
adagrad = np.zeros([dim, 1])
eps = 0.000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)
    if(t%100 == 0):
        print(str(t) + ':' + str(loss))
    gradient = 2*np.dot(x.transpose(), np.dot(x, w) - y)
    adagrad += gradient**2
    w = w - learning_rate * gradient/np.sqrt(adagrad + eps)
np.save('weight.npy', w)


ValueError: shapes (5652,164) and (163,1) not aligned: 164 (dim 1) != 163 (dim 0)

testing

In [30]:
testdata = pd.read_csv('data/test.csv', header=None, encoding='big5')
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18*9], dtype=float)
for i in range(240):
    test_x[i, :] = test_data[18*i:18*(i+1), :].reshape(1, -1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] -mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis=1).astype(float)

FileNotFoundError: [Errno 2] No such file or directory: 'data/test.csv'

prediction

In [ ]:
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)

import csv

with open('submit.csv', mode='w', newline='') as submit_files:
    csv_writer = csv.writer(submit_files)
    header = ['id', 'value']
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)

